In [447]:
import numpy as np
import os
import pandas as pd
import pymysql
import re

In [149]:
#현재 sql에서 데이터를 가져올 경우, 특수문자가 다르게 인식되어 미리 바꿔줍니다.
def processing(x):
    if '（' in x: x = re.sub('（', '(', x)
    if '）' in x: x = re.sub('）', ')', x)
    if '＋' in x: x = re.sub('＋', '+', x)
    return x

In [45]:
#단어간의 유사도를 구별하기 위한 함수입니다. 
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [340]:
#메뉴명에서 특수문자(하트, 괄호, +기호...), 세트, or가 들어있을 경우 평가에서 빼줍니다.
def check(x):
    stopwords = re.compile(".*[^ㄱ-ㅎ가-힣0-9a-z ]|.*[세셋]트|.*[set]|.*or", re.I)
    if stopwords.match(x) != None: return 0
    else: return 1

In [103]:
df = pd.read_csv('/Users/jayyoon/Desktop/yogiyo/popular_menu.csv', encoding = 'utf-8')
df.head()

,id,category_id,name,id.1,name.1,category_id.1
0,353,6,네네치킨,28814,소이크런치윙봉,6
1,353,6,네네치킨,28818,소이크런치콤보＋네네연유볼,6
2,353,6,네네치킨,28819,소이크런치윙봉＋네네연유볼,6
3,353,6,네네치킨,28820,소이크런치콤보,6
4,353,6,네네치킨,28821,치킨이 떡볶이라면（치즈）,6


In [193]:
len(df)

51936

In [157]:
df['name'] = df['name'].apply(lambda x: processing(x))
df['name.1'] = df['name.1'].apply(lambda x: processing(x))

In [412]:
recomm_df = pd.DataFrame(columns = ['restaurant_id' ,'restaurant', 'name_id', 'name', 'recomm_yn'])

,restaurant_id,restaurant,name_id,name,recomm_yn


In [413]:
recomm_df['restaurant_id'] = df['id']
recomm_df['restaurant'] = df['name']
recomm_df['name_id'] = df['id.1']
recomm_df['name'] = df['name.1']

,restaurant_id,restaurant,name_id,name,recomm_yn
0,353,네네치킨,28814,소이크런치윙봉,NaN
1,353,네네치킨,28818,소이크런치콤보+네네연유볼,NaN
2,353,네네치킨,28819,소이크런치윙봉+네네연유볼,NaN
3,353,네네치킨,28820,소이크런치콤보,NaN
4,353,네네치킨,28821,치킨이 떡볶이라면(치즈),NaN
...,...,...,...,...,...
51931,14680,중국집면사무소,1581025,짜장면,NaN
51932,14680,중국집면사무소,1581033,셋트(등심탕수육+짜장1+짬뽕1),NaN
51933,14680,중국집면사무소,1581040,짬뽕,NaN
51934,14680,중국집면사무소,1581046,간짜장,NaN


In [414]:
recomm_df = recomm_df.drop_duplicates().reset_index(drop = True)

,restaurant_id,restaurant,name_id,name,recomm_yn
0,353,네네치킨,28814,소이크런치윙봉,NaN
1,353,네네치킨,28818,소이크런치콤보+네네연유볼,NaN
2,353,네네치킨,28819,소이크런치윙봉+네네연유볼,NaN
3,353,네네치킨,28820,소이크런치콤보,NaN
4,353,네네치킨,28821,치킨이 떡볶이라면(치즈),NaN
...,...,...,...,...,...
51931,14680,중국집면사무소,1581025,짜장면,NaN
51932,14680,중국집면사무소,1581033,셋트(등심탕수육+짜장1+짬뽕1),NaN
51933,14680,중국집면사무소,1581040,짬뽕,NaN
51934,14680,중국집면사무소,1581046,간짜장,NaN


In [415]:
recomm_df['recomm_yn'] = recomm_df['name'].apply(lambda x: check(x))

,restaurant_id,restaurant,name_id,name,recomm_yn
0,353,네네치킨,28814,소이크런치윙봉,1
1,353,네네치킨,28818,소이크런치콤보+네네연유볼,0
2,353,네네치킨,28819,소이크런치윙봉+네네연유볼,0
3,353,네네치킨,28820,소이크런치콤보,1
4,353,네네치킨,28821,치킨이 떡볶이라면(치즈),0
...,...,...,...,...,...
51931,14680,중국집면사무소,1581025,짜장면,1
51932,14680,중국집면사무소,1581033,셋트(등심탕수육+짜장1+짬뽕1),0
51933,14680,중국집면사무소,1581040,짬뽕,1
51934,14680,중국집면사무소,1581046,간짜장,1


In [416]:
#메뉴 이름이 더 짧은것만을 평가하기 위해, 단어 길이를 추가해줍니다.
recomm_df['length'] = recomm_df['name'].apply(lambda x: len(x))

,restaurant_id,restaurant,name_id,name,recomm_yn,length
0,353,네네치킨,28814,소이크런치윙봉,1,7
1,353,네네치킨,28818,소이크런치콤보+네네연유볼,0,13
2,353,네네치킨,28819,소이크런치윙봉+네네연유볼,0,13
3,353,네네치킨,28820,소이크런치콤보,1,7
4,353,네네치킨,28821,치킨이 떡볶이라면(치즈),0,13
...,...,...,...,...,...,...
51931,14680,중국집면사무소,1581025,짜장면,1,3
51932,14680,중국집면사무소,1581033,셋트(등심탕수육+짜장1+짬뽕1),0,17
51933,14680,중국집면사무소,1581040,짬뽕,1,2
51934,14680,중국집면사무소,1581046,간짜장,1,3


In [418]:
recomm_df[recomm_df['recomm_yn'] == 1].sort_values(['length'], ascending = False).reset_index(drop = True)[:40]

,restaurant_id,restaurant,name_id,name,recomm_yn,length
0,4373,옥사장집밥,251969,석쇠 불고기 주는 가마솥 김치찌개 혼밥 1인 스페셜,1,28
1,739,까사꼬숑하몽과치즈샤퀴테리전문점,39884,바로 썰어 드리는 하모하몽 하몽디고타야 핸드카빙,1,26
2,11754,두목수제돈카츠,1124663,더 투툼한 수제 매콤카츠 함바그 250g이상,1,24
3,10188,이태리이야기,990497,토마토 베이스와 페페로니 토핑 페페로니 피자,1,24
4,2815,가장맛있는족발,186152,족발과 보쌈을 한꺼번에 즐기는 가족의 족보,1,23
5,3640,오로지백반,215578,돼지불고기 가득 반찬8 밥3 국3 3인분,1,22
6,3640,오로지백반,215575,돼지불고기 듬뿍 반찬8 밥2 국2 2인분,1,22
7,13233,파파존스,1420812,아이리쉬 포테이토 오리지널 크러스트 L,1,21
8,9286,파파존스,918576,아이리쉬 포테이토 오리지널 크러스트 R,1,21
9,13314,파파존스,1431041,아이리쉬 포테이토 오리지널 크러스트 L,1,21


평가란에서 최대 단어 길이기 19?정도인것 같은데 이 이상되는 단어들은 걸러줘도 될 것 같습니다.

In [419]:
recomm_df[recomm_df['recomm_yn'] == 1]

,restaurant_id,restaurant,name_id,name,recomm_yn,length
0,353,네네치킨,28814,소이크런치윙봉,1,7
3,353,네네치킨,28820,소이크런치콤보,1,7
10,434,네네치킨,34949,소이크런치윙봉,1,7
13,434,네네치킨,34956,소이크런치콤보,1,7
20,488,돈까스브라더스,36353,치즈카츠 정식,1,7
...,...,...,...,...,...,...
51925,14678,우리들의돈까스&초밥&덮밥,1580845,치즈돈까스,1,5
51931,14680,중국집면사무소,1581025,짜장면,1,3
51933,14680,중국집면사무소,1581040,짬뽕,1,2
51934,14680,중국집면사무소,1581046,간짜장,1,3


In [420]:
#두 단어간의 유사도를 구해서, 0.6이 넘을 경우, 비슷한 메뉴라고 판단하여 메뉴 이름이 더 짧은 메뉴만 평가에 추가해줍니다.
#메뉴 길이가 짧은 메뉴만 평가에 추가하는 이유는, 뿌링클과 뿌링클콤보가 있을 경우, 뿌링클이 더 평가에 적합하기 때문입니다.
try:
    
    final_df = pd.DataFrame()
    for store in recomm_df['restaurant_id'].unique():
        temp = recomm_df[recomm_df['restaurant_id'] == store].sort_values(['restaurant', 'length']).reset_index(drop = True)
        for i in range(len(temp)):
            if temp.loc[i]['recomm_yn'] == 0: continue
            for j in range(i):
                if temp.loc[j]['recomm_yn'] == 0: continue
                else:
                    if similar(temp.loc[i]['name'], temp.loc[j]['name']) >= 0.6:
                        temp.loc[i,'recomm_yn'] = 0
        final_df = pd.concat([final_df, temp])
    print('완료!')
except Exception as e:
    print(store, e)

완료!


In [426]:
#최종 평가셋 확인
final_df = final_df.drop(['length'], axis = 1)

,restaurant_id,restaurant,name_id,name,recomm_yn
0,353,네네치킨,28814,소이크런치윙봉,1
1,353,네네치킨,28820,소이크런치콤보,0
2,353,네네치킨,28818,소이크런치콤보+네네연유볼,0
3,353,네네치킨,28819,소이크런치윙봉+네네연유볼,0
4,353,네네치킨,28821,치킨이 떡볶이라면(치즈),0
...,...,...,...,...,...
0,14680,중국집면사무소,1581040,짬뽕,1
1,14680,중국집면사무소,1581025,짜장면,1
2,14680,중국집면사무소,1581046,간짜장,0
3,14680,중국집면사무소,1581052,볶음밥,1


In [440]:
final_df = final_df.reset_index(drop = True)

In [441]:
final_df.to_csv('test.csv', index = False)

In [446]:
final_df[(final_df['recomm_yn'] == 1) ]

,restaurant_id,restaurant,name_id,name,recomm_yn
0,353,네네치킨,28814,소이크런치윙봉,1
10,434,네네치킨,34949,소이크런치윙봉,1
20,488,돈까스브라더스,36353,치즈카츠 정식,1
23,489,이런피자,36443,페페로니피자 M,1
26,490,오빠네소곱창,36477,직화곱낙새볶음,1
...,...,...,...,...,...
51923,14678,우리들의돈까스&초밥&덮밥,1580835,제육덮밥,1
51925,14678,우리들의돈까스&초밥&덮밥,1580845,치즈돈까스,1
51931,14680,중국집면사무소,1581040,짬뽕,1
51932,14680,중국집면사무소,1581025,짜장면,1


# 보완점

1. 프랜차이즈의 경우, 우선 같은 프랜차이즈에 다른 지점의 같은 메뉴들은 평가셋에 넣어주었습니다. 이후 사용자의 위치가 인식되면, 가장 근접한 지점만 평가셋에 추가할 수 있게 만들어야 할 것 같습니다.

2. 단어 길이가 긴 단어들은(최대 20) 평가 메뉴에서 메뉴 이름이 전부 나타나지 않은 것으로 확인됩니다.

3. 다른 프랜차이즈에 같은 메뉴(ex: 짜장면, 아메리카노)들은 어떻게 평가항에 나오게 할지 생각해 봐야할 것 같습니다.

## SQL에 Update 

In [610]:
import pymysql
#클라우드 서버
connection = pymysql.connect(host = 'db-4afmf.pub-cdb.ntruss.com', port = 3306, user = 'mealgen',
                             passwd = "altlr!@3",
                             db ='mealgen', charset='utf8' )

In [584]:
#테스트 서버
connection = pymysql.connect(host = '127.0.0.1', port = 3306, user = 'root', passwd = 'altlrdmltleo!@', 
                     db ='mealgen', charset='utf8')

In [605]:
#database update
try:
    with connection.cursor() as cursor:
    
        sql = "UPDATE menu SET recomm_yn = %s WHERE id = %s and group_id = 2;"
        for i in range(len(final_df)):
            cursor.execute(sql, (int(final_df['recomm_yn'][i]), int(final_df['name_id'][i])))
    connection.commit()
except Exception as e:
    print(e)
finally:
    connection.close()


In [611]:
#data 가져오기
try:
    with connection.cursor() as cursor:
    
        sql = "select * from menu,restaurant \
                where menu.restaurant_id = restaurant.id \
                and menu.group_id = 2 \
                and restaurant.name not regexp 'CU|세븐일레븐|씨유|gs|홈플러스|유어스|즉시배송마켓 나우픽' ;"
        cursor.execute(sql)
        result = cursor.fetchall()
except Exception as e:
    print(e)
finally:
    connection.close()


In [617]:
result = pd.DataFrame(result)
result.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,28814,2020-07-09 00:34:19,2020-07-09 00:34:19,353,6,2,소이크런치윙봉,,[6178],18000,...,"신용카드 , 현금",1,None,Y262292,1165010300,06765,11.318777,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8a\x08...,4.65939,1
1,28818,2020-07-09 00:34:19,2020-07-09 00:34:19,353,6,2,소이크런치콤보＋네네연유볼,,[6304],21000,...,"신용카드 , 현금",1,None,Y262292,1165010300,06765,11.318777,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8a\x08...,4.65939,1
2,28819,2020-07-09 00:34:19,2020-07-09 00:34:19,353,6,2,소이크런치윙봉＋네네연유볼,,[6311],21000,...,"신용카드 , 현금",1,None,Y262292,1165010300,06765,11.318777,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8a\x08...,4.65939,1
3,28820,2020-07-09 00:34:19,2020-07-09 00:34:19,353,6,2,소이크런치콤보,,[6315],18000,...,"신용카드 , 현금",1,None,Y262292,1165010300,06765,11.318777,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8a\x08...,4.65939,1
4,28821,2020-07-09 00:34:19,2020-07-09 00:34:19,353,6,2,치킨이 떡볶이라면（치즈）,,[6318],22500,...,"신용카드 , 현금",1,None,Y262292,1165010300,06765,11.318777,b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\x8a\x08...,4.65939,1


In [618]:
backup = result[[0, 6, 16]]
backup.head()

,0,6,16
0,28814,소이크런치윙봉,1
1,28818,소이크런치콤보＋네네연유볼,0
2,28819,소이크런치윙봉＋네네연유볼,0
3,28820,소이크런치콤보,0
4,28821,치킨이 떡볶이라면（치즈）,0


In [614]:
comparison = pd.merge(backup, final_df, left_on = 0, right_on = 'name_id', how = 'inner')

In [615]:
(comparison['recomm_yn'] == comparison[16]).all()

True